In [1]:
import csv
import os
import gzip
import pandas as pd
import numpy as np

In [2]:
#path to data
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/'
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"
PATH_TO_DATA_EN = PATH_TO_DATA+"enwiki/en_parsed/"

data_time = '20180920'

In [3]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [4]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [5]:
UKWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA_UK):
    if file.endswith("art.csv"):
        UKWIKI_ART_FNMS.append(file)
UKWIKI_RED_FNMS = []
for file in os.listdir(PATH_TO_DATA_UK):
    if file.endswith("red.csv"):
        UKWIKI_RED_FNMS.append(file)   

In [6]:
print('UKWIKI_ART_FNMS:', UKWIKI_ART_FNMS)
print('UKWIKI_RED_FNMS:', UKWIKI_RED_FNMS)

UKWIKI_ART_FNMS: ['ukwiki-20180920-pages-meta-current.xml_art.csv']
UKWIKI_RED_FNMS: ['ukwiki-20180920-pages-meta-current.xml_red.csv']


In [7]:
df_uk_id_name = pd.read_csv(PATH_TO_DATA_UK+"ukwiki-" + data_time + "-id_name.csv", encoding='utf-8', sep="^")
df_uk_id_name = df_uk_id_name[['id', 'title']]
df_uk_id_name.head()

,id,title
0,3,Головна сторінка
1,13,Географія
2,584,Атом
3,585,Мільярд
4,586,Ядро


In [13]:
fn = PATH_TO_DATA_UK+UKWIKI_ART_FNMS[0]
print(fn)
df_articles = pd.read_csv(fn, encoding='UTF-8', quotechar="\"")
df_articles['link_val'] = df_articles['link_val'].str.split("#").str[0] # ???
df_articles = df_articles[df_articles['link_val'].str.strip() != '']
df_articles['link_pos_perc'] = (df_articles['link_pos']/df_articles['text_len']).round(3)    
df_articles['link_val_lc'] = df_articles['link_val'].str.lower()
df_articles.head(100)

/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/ukwiki-20180920-pages-meta-current.xml_art.csv


,id,title,text_len,link_pos,link_val,link_txt,link_pos_perc,link_val_lc
0,3,Головна сторінка,3197,0,NaN,NaN,0.000,NaN
1,13,Географія,16843,1228,Об'єкт (філософія),Об'єкт,0.073,об'єкт (філософія)
2,13,Географія,16843,1533,Ератосфен,NaN,0.091,ератосфен
3,13,Географія,16843,1599,географ,NaN,0.095,географ
4,13,Географія,16843,1611,Клавдій Птолемей,NaN,0.096,клавдій птолемей
5,13,Географія,16843,1722,Відродження,NaN,0.102,відродження
6,13,Географія,16843,1797,еллінізм,NaN,0.107,еллінізм
7,13,Географія,16843,1883,Герард Меркатор,Герарда Меркатора,0.112,герард меркатор
8,13,Географія,16843,1996,Александер фон Гумбольдт,Олександр Гумбольдт,0.119,александер фон гумбольдт
9,13,Географія,16843,2047,Карл Ріттер,NaN,0.122,карл ріттер


In [14]:
df_articles[df_articles['link_val_lc'] == 'премія «золотий глобус» за найкращий  фільм — драма']

,id,title,text_len,link_pos,link_val,link_txt,link_pos_perc,link_val_lc
15882153,2101563,Кардинал (фільм),9436,1489,Премія «Золотий глобус» за найкращий фільм — ...,найкращий драматичний фільм,0.158,премія «золотий глобус» за найкращий фільм — ...


In [15]:
df_uk_titles_series = df_articles.iloc[:,7]

In [16]:
df_uk_titles_series_stripped = df_uk_titles_series.str.replace('  ', ' ')

In [17]:
df_titles_stripped = df_uk_titles_series_stripped.to_frame()

In [18]:
#replace not stripped in df_articles with df_titles_stripped
df_articles_less = df_articles[['id', 'title', 'link_val']]

In [19]:
df_articles = pd.concat([df_articles_less, df_titles_stripped], axis=1)

In [20]:
df_articles['link_val_lc'] = df_articles['link_val_lc'].str.strip()

In [21]:
df_articles = pd.merge(df_articles, df_uk_id_name_all, how='left', left_on=['link_val_lc'], right_on=['title_lc'])

In [22]:
df_articles.head()

,id_x,title,link_val,link_val_lc,id_y,title_lc
0,3,Головна сторінка,NaN,NaN,190261.0,NaN
1,3,Головна сторінка,NaN,NaN,937102.0,NaN
2,3,Головна сторінка,NaN,NaN,1719596.0,NaN
3,13,Географія,Об'єкт (філософія),об'єкт (філософія),884280.0,об'єкт (філософія)
4,13,Географія,Ератосфен,ератосфен,21623.0,ератосфен


In [23]:
df_articles = df_articles[['id_x', 'id_y', 'link_val']]
df_articles.columns = ['id', 'link_id', 'link_val']
df_articles.head()

,id,link_id,link_val
0,3,190261.0,NaN
1,3,937102.0,NaN
2,3,1719596.0,NaN
3,13,884280.0,Об'єкт (філософія)
4,13,21623.0,Ератосфен


In [24]:
df_articles = df_articles[pd.notnull(df_articles['link_val'])]

In [25]:
df_articles[['link_id']] = df_articles[['link_id']].fillna(-1).astype(int)
df_articles['is_red_link'] = np.where(df_articles['link_id'] == -1, True, False)

In [26]:
df_articles.to_csv(PATH_TO_DATA_UK+'ukwiki-20180920-pages-links.csv', index = False, encoding='UTF-8', quotechar="\"")

In [30]:
df_id_name = pd.read_csv(PATH_TO_DATA_EN+'enwiki-' + data_time + '-id_name.csv', sep="^")

In [31]:
df_id_name.head()

,id,title
0,6197600,Flying Tigers (film)
1,6197604,Mariano Andreu
2,6197607,David Prain
3,6197617,Rut Brandt
4,6197621,Gregory C. Coleman


In [32]:
fn = PATH_TO_DATA_UK+"ukwiki-" + data_time + "-langlinks_en.csv"
df_en_langlinks = pd.read_csv(fn, encoding='UTF-16', sep="^")

In [33]:
df_en_langlinks = df_en_langlinks[pd.notnull(df_en_langlinks['ll_title'])]

In [34]:
df_en_langlinks.head()

,ll_from,ll_lang,ll_title
23,2112530,en,! (The Dismemberment Plan album)
24,1366003,en,! (disambiguation)
25,2141483,en,!!
26,425480,en,!!!
27,848226,en,!Action Pact!


In [23]:
#df_uk_en_links = pd.merge(df_en_langlinks, df_id_name, how='left', left_on=['ll_title'], right_on=['title'])

In [ ]:
# ??????

In [ ]:
df_uk_en_links = df_uk_en_links[['ll_from', 'id']]
df_uk_en_links.columns = ['id_uk', 'id_en']
df_uk_en_links[['id_uk']] = df_uk_en_links[['id_uk']].fillna(-1).astype(int)
df_uk_en_links[['id_en']] = df_uk_en_links[['id_en']].fillna(-1).astype(int)
df_uk_en_links.to_csv(PATH_TO_DATA_UK + data_time + "-langlinks_uk_en.csv", index = False, sep="^")

In [30]:
df_en_langlinks.to_csv(PATH_TO_DATA + data_time + "-langlinks_uk_en.csv", index = False, sep="^")